In [52]:
import sys
sys.path.append('../')

In [53]:
from Scripts.pinn_model import TrafficFlowForLWR_PINN, TrafficFlowForARZ_PINN
from Scripts.physics import lwr_pde_residual, physics_loss_calculator
import torch
import matplotlib.pyplot as plt
import pandas as pd

In [54]:
x_data = pd.read_csv('peachtree_dir2_lane1_10m_1s_10min.csv')

In [55]:
x = torch.tensor(x_data['x_nd'].values, dtype=torch.float32).reshape(-1,1)
t = torch.tensor(x_data['t_nd'].values, dtype=torch.float32).reshape(-1,1)
rho = torch.tensor(x_data['rho_nd'].values, dtype=torch.float32).reshape(-1,1)
u = torch.tensor(x_data['u_nd'].values, dtype=torch.float32).reshape(-1,1)
mask = torch.tensor(x_data['mask'].values, dtype=torch.bool).reshape(-1,1)

In [56]:
valid_idx = mask.squeeze(1)
x, t, rho, u = x[valid_idx], t[valid_idx], rho[valid_idx], u[valid_idx]

In [57]:
""" ### LWR Model Approximation using PINNs ###"""

lwr_model = TrafficFlowForLWR_PINN(hidden_layers=8, neurons_per_layer=20)
optimizer = torch.optim.Adam(lwr_model.parameters(), lr=0.001)

alpha = 1
beta = 1
num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Data loss on known density values only
    data_loss = torch.mean((lwr_model(x, t) - rho)**2)
    # Physics loss
    physics_loss = torch.mean(lwr_pde_residual(lwr_model, x, t)**2)
    total_loss = alpha * data_loss + beta * physics_loss

    total_loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.6f}")

Epoch 0, Loss: 0.220444
Epoch 100, Loss: 0.050913
Epoch 200, Loss: 0.040310
Epoch 300, Loss: 0.037296
Epoch 400, Loss: 0.037047
Epoch 500, Loss: 0.036931
Epoch 600, Loss: 0.036870
Epoch 700, Loss: 0.036837
Epoch 800, Loss: 0.036816
Epoch 900, Loss: 0.036810


In [58]:
# Initialize ARZ model
arz_model = TrafficFlowForARZ_PINN(hidden_layers=8, neurons_per_layer=20)
optimizer = torch.optim.Adam(arz_model.parameters(), lr=0.001)

num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()

    pred = arz_model(x, t)
    rho_pred, u_pred = pred[:, 0:1], pred[:, 1:2]

    # Data loss for density and velocity with real values
    data_loss_rho = torch.mean((rho_pred - rho)**2)
    data_loss_u = torch.mean((u_pred - u)**2)

    # Physics loss
    physics_loss = physics_loss_calculator(x, t, arz_model, beta1=1.0, beta2=1.0,
                                         gamma1=1.0, gamma2=1.0, tau=0.02)

    total_loss = data_loss_rho + data_loss_u + physics_loss

    total_loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.6f}")

Epoch 0, Loss: 284.028076
Epoch 100, Loss: 0.550987
Epoch 200, Loss: 0.499754
Epoch 300, Loss: 0.476732
Epoch 400, Loss: 0.463799
Epoch 500, Loss: 0.455871
Epoch 600, Loss: 0.450707
Epoch 700, Loss: 0.447233
Epoch 800, Loss: 0.444839
Epoch 900, Loss: 0.443141


In [59]:
### Linear Regression
from sklearn.linear_model import LinearRegression
import numpy as np

In [18]:
# Prepare data for sklearn - use .detach() to remove from computational graph
X_data = np.hstack((x.detach().numpy(), t.detach().numpy()))
rho_data = rho.detach().numpy().ravel()
u_data = u.detach().numpy().ravel()

# Train separate models for density and velocity
lr_rho = LinearRegression().fit(X_data, rho_data)
lr_u = LinearRegression().fit(X_data, u_data)

print("Linear Regression Results:")
print(f"R² score for density (ρ): {lr_rho.score(X_data, rho_data):.4f}")
print(f"R² score for velocity (u): {lr_u.score(X_data, u_data):.4f}")

Linear Regression Results:
R² score for density (ρ): 0.0028
R² score for velocity (u): 0.0158


In [ ]:
### Vanilla ANN on ARZ model
from Scripts.vanilla import VanillaMLP
vanilla_model = VanillaMLP(hidden_layers=8, neurons_per_layer=20, output_dim=2)
vanilla_optimizer = torch.optim.Adam(vanilla_model.parameters(), lr=0.001)

In [13]:

print("Training Vanilla MLP...")
for epoch in range(1000):
    vanilla_optimizer.zero_grad()
    pred = vanilla_model(x, t)
    rho_pred, u_pred = pred[:, 0:1], pred[:, 1:2]
    
    # Only data loss - no physics constraints
    loss_rho = torch.mean((rho_pred - rho)**2)
    loss_u = torch.mean((u_pred - u)**2)
    total_loss = loss_rho + loss_u
    
    total_loss.backward()
    vanilla_optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Vanilla MLP Epoch {epoch}, Loss: {total_loss.item():.6f}")

Training Vanilla MLP...
Vanilla MLP Epoch 0, Loss: 1.151428
Vanilla MLP Epoch 100, Loss: 0.173437
Vanilla MLP Epoch 200, Loss: 0.143880
Vanilla MLP Epoch 300, Loss: 0.142410
Vanilla MLP Epoch 400, Loss: 0.141470
Vanilla MLP Epoch 500, Loss: 0.140240
Vanilla MLP Epoch 600, Loss: 0.137996
Vanilla MLP Epoch 700, Loss: 0.135057
Vanilla MLP Epoch 800, Loss: 0.132480
Vanilla MLP Epoch 900, Loss: 0.129982


In [ ]:
### LSTM for ARZ model

from Scripts.lstm import LSTMNet
import torch.nn as nn

In [15]:
# Prepare sequential data for LSTM
def prepare_lstm_data(x, t, rho, u, seq_length=10):
    data = torch.cat([x, t, rho, u], dim=1)
    sequences = []
    targets = []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length, :2]  # x,t as input sequence
        target = data[i+seq_length, 2:4]  # rho, u as target
        sequences.append(seq.unsqueeze(0))
        targets.append(target.unsqueeze(0))
    
    if len(sequences) == 0:
        raise ValueError("Not enough data for sequences")
    
    sequences = torch.cat(sequences, dim=0)
    targets = torch.cat(targets, dim=0)
    return sequences, targets

In [16]:
lstm_model = LSTMNet(input_size=2, hidden_size=50, num_layers=2, output_size=2)
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("Preparing LSTM data...")
sequences, targets = prepare_lstm_data(x, t, rho, u, seq_length=10)

print("Training LSTM...")
for epoch in range(1000):
    lstm_optimizer.zero_grad()
    outputs = lstm_model(sequences)
    loss = criterion(outputs, targets)
    loss.backward()
    lstm_optimizer.step()
    
    if epoch % 100 == 0:
        print(f"LSTM Epoch {epoch}, Loss: {loss.item():.6f}")

Preparing LSTM data...
Training LSTM...
LSTM Epoch 0, Loss: 0.208867
LSTM Epoch 100, Loss: 0.089203
LSTM Epoch 200, Loss: 0.068139
LSTM Epoch 300, Loss: 0.056532
LSTM Epoch 400, Loss: 0.054961
LSTM Epoch 500, Loss: 0.050927
LSTM Epoch 600, Loss: 0.043343
LSTM Epoch 700, Loss: 0.041090
LSTM Epoch 800, Loss: 0.037508
LSTM Epoch 900, Loss: 0.033571


In [19]:
def evaluate_models(x_test, t_test, rho_test, u_test, models_dict, seq_length=10):
    """
    Evaluate all models and return metrics
    models_dict: {'model_name': model_object}
    """
    import torch.nn.functional as F
    
    results = {}
    X_test = np.hstack((x_test.detach().numpy(), t_test.detach().numpy()))
    
    for name, model in models_dict.items():
        if name == 'Linear Regression':
            rho_pred = torch.tensor(model[0].predict(X_test), dtype=torch.float32).reshape(-1,1)
            u_pred = torch.tensor(model[1].predict(X_test), dtype=torch.float32).reshape(-1,1)
            
        elif name == 'LSTM':
            # Prepare sequential test data for LSTM
            test_data = torch.cat([x_test, t_test, rho_test, u_test], dim=1)
            test_sequences = []
            test_targets = []
            
            for i in range(len(test_data) - seq_length):
                seq = test_data[i:i+seq_length, :2]  # x,t
                target = test_data[i+seq_length, 2:4]  # rho, u
                test_sequences.append(seq.unsqueeze(0))
                test_targets.append(target.unsqueeze(0))
            
            if len(test_sequences) == 0:
                print(f"Warning: Not enough test data for LSTM sequences")
                continue
                
            test_sequences = torch.cat(test_sequences, dim=0)
            test_targets = torch.cat(test_targets, dim=0)
            
            with torch.no_grad():
                lstm_pred = model(test_sequences)
                rho_pred = lstm_pred[:, 0:1]
                u_pred = lstm_pred[:, 1:2]
                
            # Use corresponding targets for evaluation
            rho_test_lstm = test_targets[:, 0:1]
            u_test_lstm = test_targets[:, 1:2]
            
            # Calculate metrics for LSTM
            mse_rho = F.mse_loss(rho_pred, rho_test_lstm).item()
            mse_u = F.mse_loss(u_pred, u_test_lstm).item()
            mae_rho = F.l1_loss(rho_pred, rho_test_lstm).item()
            mae_u = F.l1_loss(u_pred, u_test_lstm).item()
            
            results[name] = {
                'MSE_rho': mse_rho,
                'MSE_u': mse_u, 
                'MAE_rho': mae_rho,
                'MAE_u': mae_u,
                'Total_MSE': mse_rho + mse_u
            }
            continue  # Skip the general evaluation below
            
        else:  # Neural networks (PINN, Vanilla MLP)
            with torch.no_grad():
                pred = model(x_test, t_test)
                rho_pred, u_pred = pred[:, 0:1], pred[:, 1:2]
        
        # Calculate metrics for non-LSTM models
        if name != 'LSTM':
            mse_rho = F.mse_loss(rho_pred, rho_test).item()
            mse_u = F.mse_loss(u_pred, u_test).item()
            mae_rho = F.l1_loss(rho_pred, rho_test).item()
            mae_u = F.l1_loss(u_pred, u_test).item()
            
            results[name] = {
                'MSE_rho': mse_rho,
                'MSE_u': mse_u, 
                'MAE_rho': mae_rho,
                'MAE_u': mae_u,
                'Total_MSE': mse_rho + mse_u
            }
    
    return results


In [60]:
# Updated usage after training all models:
models_to_compare = {
    'PINN_ARZ': arz_model,  # Your trained ARZ model
    'Vanilla_MLP': vanilla_model,
    'LSTM': lstm_model,  # Add your trained LSTM model
    'Linear Regression': (lr_rho, lr_u)
}

# Split your data for testing (use last 20% as test)
test_size = int(0.2 * len(x))
x_test, t_test = x[-test_size:], t[-test_size:]
rho_test, u_test = rho[-test_size:], u[-test_size:]

comparison_results = evaluate_models(x_test, t_test, rho_test, u_test, models_to_compare, seq_length=10)

# Print results
print("=" * 60)
print("MODEL COMPARISON RESULTS")
print("=" * 60)

for model_name, metrics in comparison_results.items():
    print(f"\n{model_name}:")
    print("-" * 40)
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.6f}")

MODEL COMPARISON RESULTS

PINN_ARZ:
----------------------------------------
  MSE_rho: 0.139779
  MSE_u: 0.298854
  MAE_rho: 0.294851
  MAE_u: 0.428265
  Total_MSE: 0.438632

Vanilla_MLP:
----------------------------------------
  MSE_rho: 0.057946
  MSE_u: 0.096258
  MAE_rho: 0.193039
  MAE_u: 0.276417
  Total_MSE: 0.154204

LSTM:
----------------------------------------
  MSE_rho: 0.031319
  MSE_u: 0.033264
  MAE_rho: 0.141113
  MAE_u: 0.142298
  Total_MSE: 0.064583

Linear Regression:
----------------------------------------
  MSE_rho: 0.092448
  MSE_u: 0.116436
  MAE_rho: 0.234909
  MAE_u: 0.307580
  Total_MSE: 0.208884


**We can clearly see that PINNs have performed relatively poorer wrt the established models, however, these numbers are evalueated on training data solely. Now, we will extract a test dataset and will validate the models based on the unseen data**

In [21]:
import requests
import pandas as pd

In [22]:
BASE_URL = "https://data.transportation.gov/resource/8ect-6jqj.json"

In [23]:

def fetch_ngsim_api(location=None, direction=None, lane=None,
                    t_start_ms=None, t_end_ms=None,
                    limit=50000, max_pages=20, app_token=None):
    headers = {}
    if app_token:
        headers["X-App-Token"] = app_token

    where_clauses = []
    if location:  where_clauses.append(f"location='{location}'")
    if direction is not None: where_clauses.append(f"direction='{direction}'")
    if lane is not None:      where_clauses.append(f"lane_id='{lane}'")
    if t_start_ms is not None: where_clauses.append(f"global_time >= '{t_start_ms}'")
    if t_end_ms is not None:   where_clauses.append(f"global_time <= '{t_end_ms}'")
    where = " AND ".join(where_clauses) if where_clauses else None

    params = {
        "$select": "vehicle_id,frame_id,global_time,global_x,global_y,local_x,local_y,v_vel,lane_id,direction,location",
        "$order": "global_time ASC",
        "$limit": limit
    }
    if where:
        params["$where"] = where

    frames = []
    offset = 0
    for _ in range(max_pages):
        params["$offset"] = offset
        r = requests.get(BASE_URL, params=params, headers=headers, timeout=60)
        r.raise_for_status()
        chunk = r.json()
        if not chunk:
            break
        frames.append(pd.DataFrame(chunk))
        offset += limit
    if not frames:
        return pd.DataFrame()
    df = pd.concat(frames, ignore_index=True)

    # Types & units
    to_float = ["global_time","global_x","global_y","local_x","local_y","v_vel"]
    to_int   = ["frame_id","lane_id","direction"]
    for c in to_float:
        if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
    for c in to_int:
        if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

    # Time to seconds
    if "global_time" in df and df["global_time"].max() > 1e6:
        df["t"] = df["global_time"] / 1000.0
    else:
        # Fallback from frame_id (10Hz)
        df["t"] = df["frame_id"] / 10.0

    # Rename for consistency; use Global_X as longitudinal x
    df = df.rename(columns={
        "global_x":"x", "v_vel":"speed", "lane_id":"lane",
        "direction":"dir", "location":"loc"
    })

    # mph -> m/s if needed (this dataset looks like mph)
    if df["speed"].dropna().median() > 70:
        df["speed"] = df["speed"] * 0.44704

    return df[["vehicle_id","t","x","speed","lane","dir","loc"]]


In [24]:
# Choose a 5-minute window again
t0_ms = 1163050000 + 15*60*1000
t1_ms = 1163050000 + 20*60*1000

df = fetch_ngsim_api(location="peachtree", direction=2, lane=1,
                     t_start_ms=t0_ms, t_end_ms=t1_ms, limit=50000)
print(df.head(), df.shape)


  vehicle_id          t            x  speed  lane  dir        loc
0       1473  1163950.0  2230837.798  24.71     1    2  peachtree
1       1476  1163950.0  2230825.803  22.89     1    2  peachtree
2       1372  1163950.0  2230843.930  35.96     1    2  peachtree
3       1543  1163950.0  2230831.653  26.33     1    2  peachtree
4       1445  1163950.0  2230617.723  47.13     1    2  peachtree (4935, 7)


In [25]:
def aggregate_to_grid_df(df, minutes=10, dx=10.0, dt=1.0):
    # Ensure we have at least some rows
    if df.empty:
        raise ValueError("No data after API filtering")

    # Time slice to exactly minutes duration from min t
    t0 = df["t"].min()
    t1 = t0 + 60*minutes
    df = df[(df["t"]>=t0) & (df["t"]<=t1)].copy()

    # Spatial slice (trim outliers)
    x_min = df["x"].quantile(0.01)
    x_max = df["x"].quantile(0.99)
    df = df[df["x"].between(x_min, x_max)]

    # Define bins and centers
    x_bins = np.arange(x_min, x_max+dx, dx)
    t_bins = np.arange(t0, t1+dt, dt)
    x_c = (x_bins[:-1] + x_bins[1:]) / 2
    t_c = (t_bins[:-1] + t_bins[1:]) / 2

    Ix, Jt = len(x_c), len(t_c)
    rho = np.full((Ix, Jt), np.nan)
    u   = np.full((Ix, Jt), np.nan)

    for j,(ta,tb) in enumerate(zip(t_bins[:-1], t_bins[1:])):
        df_t = df[(df["t"]>=ta)&(df["t"]<tb)]
        if df_t.empty: continue
        idx = np.digitize(df_t["x"].values, x_bins) - 1
        m = (idx>=0) & (idx<Ix)
        idx = idx[m]; spd = df_t["speed"].values[m]
        counts = np.bincount(idx, minlength=Ix)
        sums   = np.bincount(idx, weights=spd, minlength=Ix)
        with np.errstate(invalid="ignore"):
            u[:,j]   = np.where(counts>0, sums/counts, np.nan)
            rho[:,j] = np.where(counts>0, counts/dx, np.nan)  # veh/m

    mask = ~np.isnan(rho)
    rho_f = np.where(mask, rho, 0.0)
    u_f   = np.where(mask, u,   0.0)

    rho_max = np.nanpercentile(rho, 99)
    v_free  = np.nanpercentile(u, 95)
    rho_nd  = np.clip(rho_f/rho_max, 0, 1)
    u_nd    = np.clip(u_f/v_free,    0, 1)

    x_nd = (x_c - x_c.min())/(x_c.max()-x_c.min())
    t_nd = (t_c - t_c.min())/(t_c.max()-t_c.min())
    Xg, Tg = np.meshgrid(x_nd, t_nd, indexing="ij")

    tensors = {
        "x": torch.tensor(Xg.reshape(-1,1), dtype=torch.float32),
        "t": torch.tensor(Tg.reshape(-1,1), dtype=torch.float32),
        "rho": torch.tensor(rho_nd.reshape(-1,1), dtype=torch.float32),
        "u": torch.tensor(u_nd.reshape(-1,1), dtype=torch.float32),
        "mask": torch.tensor(mask.reshape(-1,1), dtype=torch.float32),
        "rho_max": float(rho_max),
        "v_free": float(v_free),
        "x_centers": x_c, "t_centers": t_c
    }
    return tensors


In [ ]:
tensors2 = aggregate_to_grid_df(df, minutes=10, dx=10.0, dt=1.0)

In [34]:
x_val = tensors2["x"].numpy().ravel()
t_val = tensors2["t"].numpy().ravel()
rho_val = tensors2["rho"].numpy().ravel()
u_val = tensors2["u"].numpy().ravel()
mask_val = tensors2["mask"].numpy().ravel().astype(np.int32)

In [39]:
x_val = torch.tensor(x_val, dtype=torch.float32).reshape(-1,1)
t_val = torch.tensor(t_val, dtype=torch.float32).reshape(-1,1)
rho_val = torch.tensor(rho_val, dtype=torch.float32).reshape(-1,1)
u_val = torch.tensor(u_val, dtype=torch.float32).reshape(-1,1)
mask_val = torch.tensor(mask_val, dtype=torch.bool).reshape(-1,1)

In [40]:
valid_idx_2 = mask_val.squeeze(1)
x_val, t_val, rho_val, u_val = x_val[valid_idx_2], t_val[valid_idx_2], rho_val[valid_idx_2], u_val[valid_idx_2]

In [61]:
# Evaluate all models on validation data
models_to_compare_val = {
    'PINN_ARZ': arz_model,  # Your trained ARZ model
    'Vanilla_MLP': vanilla_model,
    'LSTM': lstm_model,
    'Linear Regression': (lr_rho, lr_u)
}

validation_results = evaluate_models(x_val, t_val, rho_val, u_val, models_to_compare_val, seq_length=10)

print("=" * 60)
print("VALIDATION MODEL COMPARISON RESULTS")
print("=" * 60)

for model_name, metrics in validation_results.items():
    print(f"\n{model_name}:")
    print("-" * 40)
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.6f}")


VALIDATION MODEL COMPARISON RESULTS

PINN_ARZ:
----------------------------------------
  MSE_rho: 0.080758
  MSE_u: 0.477265
  MAE_rho: 0.191039
  MAE_u: 0.575047
  Total_MSE: 0.558023

Vanilla_MLP:
----------------------------------------
  MSE_rho: 0.063808
  MSE_u: 0.151488
  MAE_rho: 0.173839
  MAE_u: 0.335677
  Total_MSE: 0.215296

LSTM:
----------------------------------------
  MSE_rho: 0.075076
  MSE_u: 0.120174
  MAE_rho: 0.178074
  MAE_u: 0.263052
  Total_MSE: 0.195250

Linear Regression:
----------------------------------------
  MSE_rho: 0.064216
  MSE_u: 0.164253
  MAE_rho: 0.193679
  MAE_u: 0.378048
  Total_MSE: 0.228469


In [70]:
# Only calculate physics residual for PINN models
x_val.requires_grad_(True)
t_val.requires_grad_(True)

# Calculate physics residual only for PINN
pinn_phys_residual = torch.mean(physics_loss_calculator(x_val, t_val, arz_model)**2).item()

print(f"\nPhysics Residual (Mean Squared PDE violation) on Validation Data:")
print(f"PINN_ARZ: {pinn_phys_residual:.6f}")



Physics Residual (Mean Squared PDE violation) on Validation Data:
PINN_ARZ: 0.000076


**As we can see, the physics loss incurred is a very small number, i.e the model predicts the physics behind the phenomena appropriately, while this largely points towards the other non-physics model being physically somehwat inaccurate, most likely picking up noise while training. This thereby proves that the PINNs have performed accurately while having slightly higher losses.**